In [1]:
import pandas as pd, datetime, ast, numpy as np, telebot, json, requests, os, sys, pymysql
module_path = os.path.abspath(os.path.join('/home/ktereshin/yandex/arcadia/cloud/analytics/python/work'))
if module_path not in sys.path:
    sys.path.append(module_path)
from nile.api.v1 import (
    clusters,
    aggregators as na,
    extractors as ne,
    filters as nf,
    Record
)
from vault_client import instances

from global_variables import (
    metrika_clickhouse_param_dict,
    cloud_clickhouse_param_dict
)

In [2]:
def date_range_by_weeks(start_str, end_str):
    start = datetime.datetime.strptime(start_str, '%Y-%m-%d')
    end = datetime.datetime.strptime(end_str, '%Y-%m-%d')
    delta = int((end - start).days/7) + 1
    date_list = []
    
    for i in range(delta):
        date_list.append( str((start + datetime.timedelta(days = i*7)).date()) )
        
    return date_list

def get_last_not_empty_table(folder_path):
    tables_list = sorted([folder_path + '/' + x for x in job.driver.list(folder_path)], reverse=True)
    last_table_rows = 0
    last_table = ''
    for table in tables_list:
        try:
            table_ = job.driver.read(table)
        except:
            continue
        
        if table_.row_count > last_table_rows:
            last_table_rows =  table_.row_count
            last_table = table
    if last_table:
        return last_table
    else:
        return tables_list[0]

def convert_epoch_to_end_day(epoch):
    try:
        return str(datetime.datetime.fromtimestamp(int(epoch)).replace(hour=23).replace(minute=59).replace(second=59))
    except:
        return None
    
def convert_epoch_to_date(epoch):
    try:
        return str(datetime.datetime.fromtimestamp(int(epoch)).date())
    except:
        return None
    
def date_range_by_days(start_str, end_str):
    start = datetime.datetime.strptime(start_str, '%Y-%m-%d')
    end = datetime.datetime.strptime(end_str, '%Y-%m-%d')
    delta = int((end - start).days) + 1
    date_list = []
    
    for i in range(delta):
        date_list.append( str((start + datetime.timedelta(days = i)).date()) )
        
    return date_list

In [16]:
def date_range_by_days(start_str, end_str):
    start = datetime.datetime.strptime(start_str, '%Y-%m-%d')
    end = datetime.datetime.strptime(end_str, '%Y-%m-%d')
    delta = int((end - start).days) + 1
    date_list = []
    
    for i in range(delta):
        date_list.append( str((start + datetime.timedelta(days = i)).date()) )
        
    return date_list


In [9]:
temp = {}
for date in date_range_by_days('2019-01-01', '2019-01-08'):
    temp[date] = 1

In [12]:
for date in sorted(temp):
    print(date,temp[date])

('2019-01-01', 1)
('2019-01-02', 1)
('2019-01-03', 1)
('2019-01-04', 1)
('2019-01-05', 1)
('2019-01-06', 1)
('2019-01-07', 1)
('2019-01-08', 1)


In [3]:
client = instances.Production()
yt_creds = client.get_version('ver-01d33pgv8pzc7t99s3egm24x47')
crm_sql_creds = client.get_version('ver-01d3ktedjm6ptsvwf1xq161hwk')
metrika_creds = client.get_version('ver-01d2z36msatt9mp9pcfptezksp')
yc_ch_creds = client.get_version('ver-01d2z39xj02xw7gqvv9wq757ne')

cluster = clusters.yt.Hahn(
    token = yt_creds['value']['token'],
    pool = yt_creds['value']['pool']
)
metrika_clickhouse_param_dict['user'] = metrika_creds['value']['login']
metrika_clickhouse_param_dict['password'] = metrika_creds['value']['pass']

cloud_clickhouse_param_dict['user'] = yc_ch_creds['value']['login']
cloud_clickhouse_param_dict['password'] = yc_ch_creds['value']['pass']

In [4]:
job = cluster.job()
path = get_last_not_empty_table('//home/logfeller/logs/yc-billing-export-billing-accounts-history/1h')

In [25]:
def get_ba_history(groups):
    for key, records in groups:
        
        rec_list = list(records)
        result_dict = {}
        start_date = ''
        for rec in rec_list:
            rec_ = rec.to_dict()
            rec_['date'] = convert_epoch_to_date(rec_['updated_at'])
            if not start_date:
                start_date = rec_['date']
                
            result_dict[rec_['date']] = rec_
        
        date_range = date_range_by_days(start_date, str(datetime.date.today()))
        res = {}
        for date in date_range:
            
            for date_ in sorted(result_dict):
                
                if date_ <= date:
                    result_dict[date_]['date'] = date
                    res = result_dict[date_]
                else:
                    break
            
            yield Record(key, **res)

In [26]:
job = cluster.job()
job.table(path) \
.groupby(
    'billing_account_id'
) \
.sort(
    'updated_at'
) \
.reduce(
    get_ba_history
) \
.put('//home/cloud_analytics_test/cooking_cubes/acquisition_cube/sources/ba_hist')
job.run()

VBox()

2019-04-08 13:55:28,156	INFO	Proxy n0201-sas.hahn.yt.yandex.net unbanned
2019-04-08 13:55:37,628	INFO	Operation started: http://hahn.yt.yandex.net/?page=operation&mode=detail&id=7981247b-48b6f72a-3fe03e8-6fca7751&tab=details
2019-04-08 13:57:17,658	INFO	Operation started: http://hahn.yt.yandex.net/?page=operation&mode=detail&id=52bb7024-59437fe6-3fe03e8-812b2c90&tab=details


In [ ]:
paths_dict_test = {
    'calls': '//home/cloud_analytics_test/cooking_cubes/crm_leads/calls',
    'leads_history': '//home/cloud_analytics_test/cooking_cubes/crm_leads/leads_history',
    'event_cube': '//home/cloud_analytics_test/cubes/acquisition_cube/cube',
    'crm_leads_cube': '//home/cloud_analytics_test/cubes/crm_leads/cube'
}
paths_dict_prod = {
    'calls': '//home/cloud_analytics/cooking_cubes/crm_leads/calls',
    'leads_history': '//home/cloud_analytics/cooking_cubes/crm_leads/leads_history',
    'event_cube': '//home/cloud_analytics/cubes/acquisition_cube/cube',
    'crm_leads_cube': '//home/cloud_analytics/cubes/crm_leads/cube'
}

mode = 'test'
if mode == 'test':
    paths_dict = paths_dict_test
elif mode == 'prod':
    paths_dict = paths_dict_prod

In [ ]:
sales_dict = {
    'golubin': 'enterprise',
    'dtishin': 'enterprise',
    'mpolikarpova': 'isv',
    'mvoytko': 'isv',
    'ntkachev': 'isv',
    'dtishin': 'enterprise',
    'mashmellow': 'yandex',
    'glebmarkevich': 'mass',
    'andreigusev': 'isv'
}

In [ ]:
sales_dict = {
    'dmtroe': 'mass',
    'la-n': 'mass',
    'anbalu': 'mass',
    'glebmarkevich': 'csm',
    'golubin': 'enterprise',
    'niktk': 'isv',
    'voytko': 'isv',
    'datishin': 'enterprise',
    'mblagutin': 'mass',
    'andreigusev': 'isv',
    'marinapolik': 'isv',
    'timag': 'mass',
    'mashmellow': 'yandex'
}

In [ ]:
cnx = pymysql.connect(user='readonly', password=crm_sql_creds['value']['readonly'],
                              host='percona.prod.ya-cloud-crm.stable.qloud-d.yandex.net',
                              port = 5678,
                              database='cloud8'
                             )

query = '''
SELECT
    IFNULL(accounts.id,'') account_id,
    IFNULL(accounts.ba_id,'') billing_account_id,
    accounts.date_entered account_created_at,
    IFNULL(accounts.name,'') account_name,
    IFNULL(accounts.description,'') account_description,
    IFNULL(accounts.account_type,'') account_type,
    IFNULL(l1.status,'') account_status,
    IFNULL(l1.user_name,'') sales_name,
    IFNULL(accounts.segment,'') accounts_segment
FROM accounts
INNER JOIN users l1
ON accounts.assigned_user_id=l1.id AND l1.deleted=0
WHERE
     accounts.deleted=0 
'''

accounts = pd.read_sql_query(query, cnx)
cnx.close()

In [ ]:
accounts[accounts['billing_account_id'] != '']

In [ ]:
accounts['sales_name'].unique()

In [4]:
cnx = pymysql.connect(user='readonly', password=crm_sql_creds['value']['readonly'],
                              host='percona.prod.ya-cloud-crm.stable.qloud-d.yandex.net',
                              port = 5678,
                              database='cloud8'
                             )

query = '''
SELECT
    acc_hist.parent_id as account_id,
    date_created,
    after_value_string as assigned_by,
    IFNULL(l1.user_name,'') sales_name,
    acc.segment as account_segment,
    IFNULL(acc.name,'') account_name
FROM accounts_audit as acc_hist
LEFT JOIN users l1
ON acc_hist.after_value_string=l1.id AND l1.deleted=0
LEFT JOIN accounts as acc
ON acc_hist.parent_id=acc.id AND acc.deleted=0
WHERE
    field_name = 'assigned_user_id'
'''

test = pd.read_sql_query(query, cnx)
cnx.close()

In [36]:
cnx = pymysql.connect(user='readonly', password=crm_sql_creds['value']['readonly'],
                              host='percona.prod.ya-cloud-crm.stable.qloud-d.yandex.net',
                              port = 5678,
                              database='cloud8'
                             )

query = '''
SELECT
    acc.ba_id as billing_account_id,
    acc_hist.parent_id as account_id,
    CAST(DATE_ADD(date_created, INTERVAL -3 HOUR) AS CHAR) as event_time,
    'changes_sales' as event,
    IFNULL(l1.user_name,'') sales_name
FROM accounts_audit as acc_hist
LEFT JOIN users l1
ON acc_hist.after_value_string=l1.id AND l1.deleted=0
LEFT JOIN accounts as acc
ON acc_hist.parent_id=acc.id AND acc.deleted=0
WHERE
    acc.ba_id IS NOT NULL
    AND field_name = 'assigned_user_id'
'''

account_changes = pd.read_sql_query(query, cnx)
cnx.close()

In [37]:
account_changes

,billing_account_id,account_id,event_time,event,sales_name
0,dn25fsuvlecgrcj6gm6t,ff82dd7e-23b8-11e9-8290-ba4d300fecbc,2019-01-29 08:28:24,changes_sales,glebmarkevich
1,dn2nh72cqm0vbrircdk9,02297bbc-547b-11e9-b4f8-7ac6a7523101,2019-04-01 09:38:08,changes_sales,dmtroe
2,dn24rksev9on2bb8jqd0,024e5f96-5083-11e9-a310-e0fc2c762fc2,2019-03-27 08:25:19,changes_sales,la-n
3,dn2i85qaju401hoq3if4,046f6e86-4c97-11e9-8946-d4197eff0c7e,2019-03-22 08:38:28,changes_sales,la-n
4,dn2tmas5odfq7h0h4ms1,0783a5a4-496f-11e9-a097-8e64c482fb7d,2019-03-18 08:14:41,changes_sales,la-n
5,dn2rj84tqlc2p7t8fi8n,08064228-459d-11e9-9be7-1e90c927451e,2019-03-13 11:33:54,changes_sales,dmtroe
6,dn26i44mp7k1ab9kbisp,09941eca-4b07-11e9-9da1-e225f8fec9aa,2019-03-20 08:55:19,changes_sales,la-n
7,dn22ron6bs90dpsngg37,0a3c0a0e-4a26-11e9-ace4-9ad2bc3ace24,2019-03-19 06:04:43,changes_sales,dmtroe
8,dn2u2lpiguafkvh35off,0ac55ae2-4b17-11e9-b0af-8b12437172c1,2019-03-20 10:49:52,changes_sales,dmtroe
9,dn2lgpv6pu597rtjrsrp,0b9eaae2-5097-11e9-9b8c-f32795bb3bdb,2019-03-27 10:48:45,changes_sales,dmtroe


In [38]:
cluster.write('//home/cloud_analytics_test/cooking_cubes/acquisition_cube/sources/account_sales_changes', account_changes)

In [ ]:
job = cluster.job()
for path in['billing_accounts_history_path']:
    valid_path = get_last_not_empty_table(paths_dict_temp[path], job)
    paths_dict[path] = valid_path

schema = {
    "balance": float,
    "balance_client_id": str,
    "balance_contract_id": str,
    "billing_account_id": str,
    "billing_threshold": str,
    "client_id": str,
    "currency": str,
    "date": str,
    "feature_flags": str,
    "iso_eventtime": str,
    "master_account_id": str,
    "metadata": str,
    "name": str,
    "owner_id": str,
    "payment_cycle_type": str,
    "payment_method_id": str,
    "payment_type": str,
    "person_type": str,
    "state": str,
    "type": str,
    "updated_at": int,
    "usage_status": str,
    "block_reason": str
}

job = cluster.job()
job.table(paths_dict['billing_accounts_history_path']) \
.project(
    ne.all(),
    block_reason = ne.custom(lambda x,y:get_reason(y) if x == 'suspended' else 'Unlocked', 'state', 'metadata')
) \
.groupby(
    'billing_account_id'
) \
.sort(
    'updated_at'
) \
.reduce(
    get_ba_history
) \
.project(
    **apply_types_in_project(schema)
) \
.put(paths_dict['ba_hist'], schema = schema)
job.run()